In [1]:
from __future__ import (division, unicode_literals, absolute_import,
                        print_function)

from ema_workbench import (Model, RealParameter, Constant, CategoricalParameter, 
                           TimeSeriesOutcome, ScalarOutcome, ema_logging, perform_experiments, 
                           MultiprocessingEvaluator, Policy)
from ema_workbench.connectors.vensim import (VensimModel , VensimModelStructureInterface, set_value)
from ema_workbench.em_framework.samplers import sample_levers
from ema_workbench.util.utilities import save_results
from ema_workbench import load_results
from ema_workbench.analysis.pairs_plotting import (pairs_lines, pairs_scatter)
from ema_workbench.analysis import clusterer, plotting, Density, prim
from ema_workbench.analysis.plotting import lines, envelopes, kde_over_time, multiple_densities



import timeit
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import itertools

from ema_workbench.analysis.plotting import lines

C:\Programs\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath

In [2]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:70% !important; }</style>"))

# Select vensim file

In [3]:
wd = r'./VENSIM' # Directory of the vensim model
model_file=r'Maasmodel_v075.vpmx' # Name of the vensim model

# General manual input

In [4]:
#FILL IN THIS CELL
#WEIR
Crest_level_weir1 = 8.35
Crest_level_weir2 = 7.15
Crest_level_weir3 = 6.9
Crest_level_weir4 = 7.2
Crest_level_weir5 = [8.4, 8.6]
Crest_level_weir6 = 10.9
Location_weir1    = 57500
Location_weir2    = 69500
Location_weir3    = 87500
Location_weir4    = 135500
Location_weir5    = [163500, 150500]
Location_weir6    = 189500
Leakage_low_weir1    = 1.2e6
Leakage_low_weir2    = 1e6
Leakage_low_weir3    = 1e6
Leakage_low_weir4    = 3e6
Leakage_low_weir5    = 3e6
Leakage_low_weir6    = 2e6
Leakage_up_weir1     = 6e6
Leakage_up_weir2     = 5e6
Leakage_up_weir3     = 5e6
Leakage_up_weir4     = 9e6
Leakage_up_weir5     = 9e6
Leakage_up_weir6     = 8e6

#LOCK
Intensity_commercial_lock1 = 44
Intensity_commercial_lock2 = 62
Intensity_commercial_lock3 = 439
Intensity_commercial_lock4 = 547
Intensity_commercial_lock5 = 289
Intensity_commercial_lock6 = 334
Intensity_leisure_lock1 = 643
Intensity_leisure_lock2 = 518
Intensity_leisure_lock3 = 458
Intensity_leisure_lock4 = 498
Intensity_leisure_lock5 = 511
Intensity_leisure_lock6 = 807

Make arrays to feed lookups

In [5]:
#NAME CLIMATE SCENARIOS
CS1 = 'Climate scenario GL'
CS2 = 'Climate scenario WH'

#SAVE FIGURE
save = False
fig_size = (10,7)
dpi = 300
%matplotlib inline

#Starting year
start_year = 2020
end_year   = 2100


#100 years
years = np.linspace(start_year,end_year,81)

#SECONDS TO Week
SEC_WEEK = 60*60*24*7 

#Different draught durations in weeks
Draught_durations = [4,6,8,10,12,14,16] #Duur van de droogte periode variërend van 4 tot 16 weken

#Different draught intensities in m3/s
Draught_intensities = [20,25,30,35,40,45,1000] #Maximale afvoer in periodes van droogte, 1000 staat voor geen droogte periode 

#Distance from inflow
distance_inflow = np.linspace(500, 189500, 190)

#100 year discharge series
year_100_dischargeseries = list(range(1911,2015))

# Model parameters

In [6]:
# turn on logging
ema_logging.log_to_stderr(ema_logging.INFO)

# instantiate a model
vensimModel = VensimModel('MaasModel', wd=wd, model_file=model_file)

# (X) External factors and States of the World
vensimModel.uncertainties = [CategoricalParameter('Climate scenario', [1, 2]),
                             CategoricalParameter('Socio Economic scenario', [1, 2]),
                            
#Uncertain model variables  
                             CategoricalParameter('Draught period intensity', Draught_intensities ),
                             CategoricalParameter('Draught period length', Draught_durations),
                             CategoricalParameter('Simulated year', years),
                             CategoricalParameter('Water loss one levelling cycle', [6000, 7000,8000,9000,10000,11000,12000]),
                             CategoricalParameter('Extra shipping intensity Diked Meuse', [0.2,0.25,0.3,0.35,0.4,0.45,0.5]),
                             CategoricalParameter('Percentage of ships that are class IV', [0.2,0.22,0.24,0.26,0.28,0.3]),
                             CategoricalParameter('Percentage of ships that are class V',[0.4,0.42,0.44,0.46,0.48,0.5]),
                             RealParameter('Percentage transport via water', 0.1, 0.3),
                             RealParameter('Leakage weir and lock[Weir1]', Leakage_low_weir1, Leakage_up_weir1),
                             RealParameter('Leakage weir and lock[Weir2]', Leakage_low_weir2, Leakage_up_weir2),
                             RealParameter('Leakage weir and lock[Weir3]', Leakage_low_weir3, Leakage_up_weir3),
                             RealParameter('Leakage weir and lock[Weir4]', Leakage_low_weir4, Leakage_up_weir4),
                             RealParameter('Leakage weir and lock[Weir5]', Leakage_low_weir5, Leakage_up_weir5),
                             RealParameter('Leakage weir and lock[Weir6]', Leakage_low_weir6, Leakage_up_weir6),
                             CategoricalParameter('Year from discharge series', year_100_dischargeseries)
                                                          ]
                                                  

# (L) Policy levers and/or design Alternatives
vensimModel.levers = [
                      CategoricalParameter('Crest level weir[Weir5]', [8.4, 8.6]),
                      CategoricalParameter('Crest level weir[Weir6]', [8.5, 8.7]),
                      CategoricalParameter('Crest level summer', [0,0.2]),
                      CategoricalParameter('Weir location[Weir5]', [163500, 150500]),
                      CategoricalParameter('Pump switch', [0 , 1]),
                      CategoricalParameter('Levelling restrictions switch', [0 , 1])
                        ]


# (R) Other system parameters
vensimModel.constants = [#Water flow
                         Constant('Crest level weir[Weir1]', 8.35),
                         Constant('Crest level weir[Weir2]', 7.15),
                         Constant('Crest level weir[Weir3]', 6.9),
                         Constant('Crest level weir[Weir4]', 7.2),                   
                         Constant('Closing retaining lock Limmel',  1200  ),
                         Constant('Weir location[Weir1]', 57500),
                         Constant('Weir location[Weir2]', 69500),
                         Constant('Weir location[Weir3]', 87500),
                         Constant('Weir location[Weir4]', 136500),
                         Constant('Weir location[Weir6]', 189500),
                         Constant('Discharge to turn on pumps Maasbracht',  50  ),
                         Constant('Dryest time of the year', 26),
                         Constant('Grid cell length', 1000),
                         Constant('Initial equilibrium depth', 3.5),
                         Constant('Low discharge', 50*SEC_WEEK),
                         Constant('Maximum critical depth', 1.5),
                         Constant('Maximum water depth', 0.2),
                         Constant('Minimum width', 1),
                         Constant('Reference year', 2014),
                         Constant('Water loss due to levelling at Maasbracht', 15*SEC_WEEK),
                         Constant('Water loss Weurt', 0.8*SEC_WEEK),
    
                        

                         #Shipping flow
                         Constant('Commercial shipping growth WLOL', -0.00041675 ),
                         Constant('Commercial shipping growth WLOH', 0.000833  ),
                         Constant('Draught ship of CEMT class III', 2.7),
                         Constant('Draught ship of CEMT class IV', 3),
                         Constant('Draught ship of CEMT class V', 3.5),
                         Constant('Intensity commercial shipping reference year[Weir1]', Intensity_commercial_lock1 ),
                         Constant('Intensity commercial shipping reference year[Weir2]', Intensity_commercial_lock2 ),
                         Constant('Intensity commercial shipping reference year[Weir3]', Intensity_commercial_lock3 ),
                         Constant('Intensity commercial shipping reference year[Weir4]', Intensity_commercial_lock4 ),
                         Constant('Intensity commercial shipping reference year[Weir5]', Intensity_commercial_lock5 ),
                         Constant('Intensity commercial shipping reference year[Weir6]', Intensity_commercial_lock6 ),
                         Constant('Intensity leisure shipping reference year[Weir1]', Intensity_leisure_lock1),
                         Constant('Intensity leisure shipping reference year[Weir2]', Intensity_leisure_lock2),
                         Constant('Intensity leisure shipping reference year[Weir3]', Intensity_leisure_lock3),
                         Constant('Intensity leisure shipping reference year[Weir4]', Intensity_leisure_lock4),
                         Constant('Intensity leisure shipping reference year[Weir5]', Intensity_leisure_lock5),
                         Constant('Intensity leisure shipping reference year[Weir6]', Intensity_leisure_lock6),
                         Constant('Leisure shipping growth WLOL', -0.000275 ),
                         Constant('Leisure shipping growth WLOH', -0.0015 ),
                         Constant('Large lock factor', 1.75),
                         Constant('Middle sized lock factor', 1.4),
                         Constant('Small lock factor', 1),
                         Constant('Max number of ships in small chamber', 4),
                         Constant('Minimum percentage of ships sailing over the river Meuse', 0.1),
                         Constant('Percentage transport via water reference year', 0.2),
                         Constant('Sill height in front of lock[Weir1]', 4.1),
                         Constant('Sill height in front of lock[Weir2]', 2.2),
                         Constant('Sill height in front of lock[Weir3]', 1.25),
                         Constant('Sill height in front of lock[Weir4]', 0.1),
                         Constant('Sill height in front of lock[Weir5]', 1.5),
                         Constant('Sill height in front of lock[Weir6]', 3),
                         Constant('Time sailing in to lock', 10),
                         Constant('Time sailing out of lock', 10),
                         Constant('Time securing the ship', 5),
    
    
                         #Fresh water buffer
                         Constant('Original crest height[Weir1]', 8.35),
                         Constant('Original crest height[Weir1]', 7.15),                         
                         Constant('Original crest height[Weir1]', 6.9),
                         Constant('Original crest height[Weir1]', 7.2),
                         Constant('Original crest height[Weir1]', 8.4),
                         Constant('Original crest height[Weir1]', 8.5),
                         
                         
                         
                         
                         Constant('Number of small chambers[Weir1]', 0),
                         Constant('Number of small chambers[Weir2]', 0),
                         Constant('Number of small chambers[Weir3]', 2),
                         Constant('Number of small chambers[Weir4]', 2),
                         Constant('Number of small chambers[Weir5]', 1),
                         Constant('Number of small chambers[Weir6]', 1),
                         Constant('Number of middle sized chambers[Weir1]', 0),
                         Constant('Number of middle sized chambers[Weir2]', 0),  
                         Constant('Number of middle sized chambers[Weir3]', 0),  
                         Constant('Number of middle sized chambers[Weir4]', 0),  
                         Constant('Number of middle sized chambers[Weir5]', 0),  
                         Constant('Number of middle sized chambers[Weir6]', 1),                       
                         Constant('Number of large chambers[Weir1]', 1),                      
                         Constant('Number of large chambers[Weir2]', 1),                      
                         Constant('Number of large chambers[Weir3]', 1),                      
                         Constant('Number of large chambers[Weir4]', 1),                       
                         Constant('Number of large chambers[Weir5]', 0),                     
                         Constant('Number of large chambers[Weir6]', 0)
                         ]

                       

                        
    
    
    
# (M) Performance Metrics and Outcomes of Interest
vensimModel.outcomes = [TimeSeriesOutcome('Percentage of full fresh water buffer[Weir1]'),
                        TimeSeriesOutcome('Percentage of full fresh water buffer[Weir2]'),
                        TimeSeriesOutcome('Percentage of full fresh water buffer[Weir3]'),
                        TimeSeriesOutcome('Percentage of full fresh water buffer[Weir4]'),
                        TimeSeriesOutcome('Percentage of full fresh water buffer[Weir5]'),
                        TimeSeriesOutcome('Percentage of full fresh water buffer[Weir6]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir1]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir2]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir3]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir4]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir5]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir6]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir1]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir2]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir3]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir4]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir5]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir6]'),
                        TimeSeriesOutcome('Days of free flow')]
                        


# Perform the experiments and save the results

In [7]:
policies = [Policy('No policy',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Crest Grave raised',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.6, 'Levelling restrictions switch': 0, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Dynamic crest level',**{'Crest level summer': 0.2, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Pumps on',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 1, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Levelling restrictions',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 1, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Relocate weir',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 0, 'Weir location[Weir5]': 150500, 'Crest level weir[Weir6]':8.5}),
Policy('Relocate weir and raise crest level Lith',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 0, 'Weir location[Weir5]': 150500, 'Crest level weir[Weir6]':8.7}),
Policy('Crest level raised and pumps on',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.6, 'Levelling restrictions switch': 0, 'Pump switch': 1, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Dynamic crest level and pumps on',**{'Crest level summer': 0.2, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 1, 'Weir location[Weir6]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Crest level raised and levelling restrictions',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.6, 'Levelling restrictions switch': 1, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Dynamic crest level and levelling restrictions',**{'Crest level summer': 0.2, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 1, 'Pump switch': 0, 'Weir location[Weir5]': 163500, 'Crest level weir[Weir6]':8.5}),
Policy('Relocate weir and pumps on',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 0, 'Pump switch': 1, 'Weir location[Weir5]': 150500, 'Crest level weir[Weir6]':8.5}),
Policy('Relocate weir and levelling restrictions',**{'Crest level summer': 0, 'Crest level weir[Weir5]': 8.4, 'Levelling restrictions switch': 1, 'Pump switch': 0, 'Weir location[Weir5]': 150500, 'Crest level weir[Weir6]':8.5})
           ]


In [8]:
nr_scenarios = 8000
nr_policies = len(policies)
nr_experiments = nr_scenarios*nr_policies


print('Simulating the model with', nr_scenarios, 'scenarios and', nr_policies, 'design decisions')

Simulating the model with 8000 scenarios and 13 design decisions


In [9]:
"""
start_time = timeit.default_timer()

#results = perform_experiments(vensimModel, nr_scenarios, nr_policies)
with MultiprocessingEvaluator(vensimModel) as evaluator:
    results = evaluator.perform_experiments(scenarios=nr_scenarios, policies=policies)


elapsed = timeit.default_timer() - start_time
print("Total time in minutes:", elapsed/60, "-- Time per run in seconds:", 
      elapsed/(nr_scenarios * max(len(policies),1)) ) 
"""      


'\nstart_time = timeit.default_timer()\n\n#results = perform_experiments(vensimModel, nr_scenarios, nr_policies)\nwith MultiprocessingEvaluator(vensimModel) as evaluator:\n    results = evaluator.perform_experiments(scenarios=nr_scenarios, policies=policies)\n\n\nelapsed = timeit.default_timer() - start_time\nprint("Total time in minutes:", elapsed/60, "-- Time per run in seconds:", \n      elapsed/(nr_scenarios * max(len(policies),1)) ) \n'

# Save results

In [10]:
#experiments, outcomes = results
#save_results(results, r'C:\Users\douwe\OneDrive\Documenten\1.0 School\8.0 Thesis\5.0 Execution\Maasmodel\Results\Policies_256000a.tar')